In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
import time

In [75]:
class Preprocess():
    
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.train_iter , self.test_iter = AG_NEWS()
        self.train_dataloader, self.test_dataloader = self.prepare_data()
        self.tokenizer = get_tokenizer("basic_english")
        self.vocab = build_vocab_from_iterator(self.yield_tokens(self.train_iter), specials=["<unk>"])
        self.vocab.set_default_index(self.vocab["<unk>"])
        self.text_pipeline = lambda x: self.vocab(self.tokenizer(x))
        self.label_pipeline = lambda x: int(x) - 1
        
    def collate_batch(self, batch):
        label_list, text_list, offsets = [], [], [0]
        for _label, _text in batch:
            label_list.append(label_pipeline(_label))
            processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
            text_list.append(processed_text)
            offsets.append(processed_text.size(0))
        label_list = torch.tensor(label_list, dtype=torch.int64)
        offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
        text_list = torch.cat(text_list)
        return label_list.to(self.device), text_list.to(self.device), offsets.to(self.device)
    
    def prepare_data(self):
        train_dataset = to_map_style_dataset(self.train_iter)
        test_dataset = to_map_style_dataset(self.test_iter)
        train_dataloader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, collate_fn=self.collate_batch
        )
        test_dataloader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=True, collate_fn=self.collate_batch
        )
        
        return train_dataloader, test_dataloader
    
    def yield_tokens(self, data_iter):
        for _, text in data_iter:
            yield self.tokenizer(text)
    

In [76]:
batch_size = 64
pre = Preprocess(batch_size)

In [64]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, num_layers, hidden_size, num_class):
        super(TextClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size, num_class)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_class)
        self.dropout = nn.Dropout(0.2)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets, h):
        x = self.embedding(text, offsets)
        lstm_out, (h, c) = self.lstm(x, h)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_size)
        lstm_out = self.dropout(lstm_out)
        lstm_out = self.fc1(lstm_out)
        lstm_out = self.fc2(lstm_out)
        return lstm_out
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        #h0 = torch.zeros((self.num_layers,batch_size,self.hidden_size)).to(device)
        #c0 = torch.zeros((self.num_layers,batch_size,self.hidden_size)).to(device)
        h0 = torch.zeros((self.num_layers, self.hidden_size)).to(device)
        c0 = torch.zeros((self.num_layers, self.hidden_size)).to(device)
        hidden = (h0,c0)
        return hidden

In [65]:
num_class = len(set([label for (label, text) in pre.train_iter]))
vocab_size = len(pre.vocab)
embedding_size = 64
num_layers = 1
hidden_size = 256
device = pre.device

model = TextClassifier(vocab_size, embedding_size, num_layers, hidden_size, num_class).to(device)
EPOCHS = 10  # epoch
lr = 0.001  # learning rate
#BATCH_SIZE = 64  # batch size for training
#batch_size = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [66]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    h = model.init_hidden(batch_size)
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        
        label = label.to(device)
        text = text.to(device)
        offsets = offsets.to(device)
        h = tuple([each.data for each in h])
        
        optimizer.zero_grad()
        predicted_label = model.forward(text, offsets, h)
        
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

In [67]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    val_h = model.init_hidden(batch_size)
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            
            label = label.to(device)
            text = text.to(device)
            offsets = offsets.to(device)
            val_h = tuple([each.data for each in val_h])
            
            predicted_label = model(text, offsets, val_h)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [69]:
total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(pre.train_dataloader)
    accu_val = evaluate(pre.test_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |   500/ 1875 batches | accuracy    0.935
| epoch   1 |  1000/ 1875 batches | accuracy    0.933
| epoch   1 |  1500/ 1875 batches | accuracy    0.931
-----------------------------------------------------------
| end of epoch   1 | time: 39.44s | valid accuracy    0.911 
-----------------------------------------------------------
| epoch   2 |   500/ 1875 batches | accuracy    0.955
| epoch   2 |  1000/ 1875 batches | accuracy    0.951
| epoch   2 |  1500/ 1875 batches | accuracy    0.950
-----------------------------------------------------------
| end of epoch   2 | time: 40.25s | valid accuracy    0.907 
-----------------------------------------------------------
| epoch   3 |   500/ 1875 batches | accuracy    0.972
| epoch   3 |  1000/ 1875 batches | accuracy    0.972
| epoch   3 |  1500/ 1875 batches | accuracy    0.972
-----------------------------------------------------------
| end of epoch   3 | time: 40.45s | valid accuracy    0.909 
-------------------------------

In [70]:
total_accu = None
for epoch in range(11, 16):
    epoch_start_time = time.time()
    train(pre.train_dataloader)
    accu_val = evaluate(pre.test_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch  11 |   500/ 1875 batches | accuracy    0.975
| epoch  11 |  1000/ 1875 batches | accuracy    0.976
| epoch  11 |  1500/ 1875 batches | accuracy    0.975
-----------------------------------------------------------
| end of epoch  11 | time: 39.47s | valid accuracy    0.910 
-----------------------------------------------------------
| epoch  12 |   500/ 1875 batches | accuracy    0.976
| epoch  12 |  1000/ 1875 batches | accuracy    0.975
| epoch  12 |  1500/ 1875 batches | accuracy    0.976
-----------------------------------------------------------
| end of epoch  12 | time: 40.06s | valid accuracy    0.908 
-----------------------------------------------------------
| epoch  13 |   500/ 1875 batches | accuracy    0.976
| epoch  13 |  1000/ 1875 batches | accuracy    0.975
| epoch  13 |  1500/ 1875 batches | accuracy    0.975
-----------------------------------------------------------
| end of epoch  13 | time: 39.80s | valid accuracy    0.908 
-------------------------------

In [72]:
print("Checking the results of test dataset.")
accu_test = evaluate(pre.test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy    0.908


In [77]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}


def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output = model(text, torch.tensor([0]), h)
        return output.argmax(1).item() + 1


ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to(device)

print("This is a %s news" % ag_news_label[predict(ex_text_str, pre.text_pipeline)])

This is a Sports news
